In [2]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

import os
import os.path as path
import sys
import re
import json
import datetime
from datetime import timedelta

from IPython.display import display, Markdown, Latex

from utils import kde_scatter, bayesian_correlation, summary

import warnings
warnings.filterwarnings("ignore", "dropping on a non-lexsorted multi-index without a level parameter may impact performance", append=True)


%matplotlib inline
%load_ext autoreload

%autoreload 2

In [7]:
sys.path.append(r"C:\Users\alfoc.ULAVAL\Desktop\OpenIMU\python")

from libopenimu.importers import OpenIMUImporter

from libopenimu.db.DBManager import DBManager
from libopenimu.models.Participant import Participant

DATAPATH = r"Z:\AlexisFC\SN\Expedtition 2018\RAW-Martine_OPEN_IMU_MONTRE_2018\OPENIMU"
sampling_rate = 50
sampling_rate_baro = 1

oi = OpenIMUImporter.OpenIMUImporter(DBManager("oi.oi"), Participant())

dfs_imu  = []
dfs_baro = []

for file in [x for x in os.listdir(DATAPATH) if '.oimu' in x][:]:
    with open(path.join(DATAPATH, file), 'rb') as f:
        data = oi.readDataFile(f)
        
    # Fix wrong timestamp
    has_corrupt_data = False
    print('file {}, timestamp : {}'.format(file, list(data.keys())))
    for key in list(data.keys()):
        if key<0:
            data.pop(key)
            has_corrupt_data = True

    for timestamp, values in data.items():
        df_imu = pd.DataFrame(data=np.array(values['imu']), columns=['ax', 'ay', 'az', 'gx', 'gy', 'gz', 'mx', 'my', 'mz'])
        df_imu.index  = pd.datetime.fromtimestamp(timestamp) + timedelta(seconds=1/sampling_rate) * df_imu.index
        df_baro = pd.DataFrame(data=np.array(values['baro']), columns=['temp', 'pressure'])
        df_baro.index = pd.datetime.fromtimestamp(timestamp) + timedelta(seconds=1/sampling_rate_baro) * df_baro.index
        
        dfs_imu.append(df_imu)
        dfs_baro.append(df_baro)
        
dfs_imu = pd.concat(dfs_imu).sort_index()
dfs_baro = pd.concat(dfs_baro).sort_index()

In [9]:
len(dfs_imu.index.day.unique())

18

In [5]:
timestamp

-993772109

In [6]:
file

'10.oimu'